In [1]:
import numpy as np
import pandas as pd
from IPython.display import display,HTML

1. we determine the place from which we extract the data we are interested in (https://adatbank.mlsz.hu/league/59/0/25413/17.html)       ---- where? what?
2. will extract the data of the 2nd division of the Hungarian championship (i'm interested in that), using a simple method, namely **pd.read_html**  ------ whit what?

                2/a. write a simple finction to this
                2/b. and we will deal with a small inconvenience

First of all, let's see how many tables our site contains

In [13]:
example_df = pd.read_html('https://adatbank.mlsz.hu/league/59/0/25413/17.html')

after checking it turns out that we need one of the 19 options

In [14]:
len(example_df)

19

after interpreting the options, let's write a usable function

In [4]:
def roundAndposition(lastRoundNum):
    df = pd.DataFrame()
    # enter the value of the last round
    for num in range(1,lastRoundNum+1,1):
        rawTable = pd.read_html(f'http://ada1bank.mlsz.hu/league?evad=59&szervezet=0&verseny=25413&fordulo={num}')
        # pd.html usually returns a list of lists (we have 19 now) - it is important to find out which one we need - we found out beforehand, in our case, number 5 is the winner
        df2 = rawTable[5][rawTable[5].columns[2:11]]
        
        df = pd.concat([df,df2])
    
    # mark the ranking with a new column
    df = (df
        .assign(Position = df.index+1))
    
    # define and sort necessary columns
    new_col = ['team','round','win','draw','lost','goalfor','goalagainst','goaldiff','point','position']
    df = (df
        .rename(columns={k:v for k,v in zip(df.columns,new_col)}))
    
    
    return df.reset_index(drop=True)

In [5]:
df = roundAndposition(17)

Don't forget to check the data, because we may be in for a surprise

        for example, let's look at the team names after such a scratch

In [6]:
df.team.unique()

array(['GYIRMÓT\xa0FC\xa0GYŐR', 'FC\xa0AJKA', 'SOROKSÁR\xa0SC',
       'KOLORCITY\xa0KAZINCBARCIKA\xa0SC', 'MTK\xa0BUDAPEST',
       'BFC\xa0SIÓFOK', 'ETO\xa0FC\xa0GYŐR', 'PMFC', 'TISZAKÉCSKEI\xa0LC',
       'AQVITAL\xa0FC\xa0CSÁKVÁR', 'SZOMBATHELYI\xa0HALADÁS',
       'SZEGED-CSANÁD\xa0GROSICS\xa0AKADÉMIA',
       'BÉKÉSCSABA\xa01912\xa0ELŐRE', 'CREDOBUS\xa0MOSONMAGYARÓVÁR',
       'DVTK', 'NYÍREGYHÁZA\xa0SPARTACUS\xa0FC',
       'HR-RENT\xa0KOZÁRMISLENY', 'DOROGI\xa0FC', 'BUDAFOKI\xa0MTE',
       'SZENTLŐRINC'], dtype=object)

In [7]:
df = (df
     .assign(team = [' '.join(x.split()) for x in df.team]))

so it will be suitable - check

In [8]:
df.team.unique()

array(['GYIRMÓT FC GYŐR', 'FC AJKA', 'SOROKSÁR SC',
       'KOLORCITY KAZINCBARCIKA SC', 'MTK BUDAPEST', 'BFC SIÓFOK',
       'ETO FC GYŐR', 'PMFC', 'TISZAKÉCSKEI LC', 'AQVITAL FC CSÁKVÁR',
       'SZOMBATHELYI HALADÁS', 'SZEGED-CSANÁD GROSICS AKADÉMIA',
       'BÉKÉSCSABA 1912 ELŐRE', 'CREDOBUS MOSONMAGYARÓVÁR', 'DVTK',
       'NYÍREGYHÁZA SPARTACUS FC', 'HR-RENT KOZÁRMISLENY', 'DOROGI FC',
       'BUDAFOKI MTE', 'SZENTLŐRINC'], dtype=object)

let's see how it works manually

In [15]:
df[(df.team == 'NYÍREGYHÁZA SPARTACUS FC') & (df['round'] == 10) ]

,team,round,win,draw,lost,goalfor,goalagainst,goaldiff,point,position
187,NYÍREGYHÁZA SPARTACUS FC,10,4,2,4,14,11,3,14,8


let's generalize it with a function

In [10]:
def posTeam(team,round_):
    return df[(df.team == team) & (df['round'] == round_) ]

In [19]:
posTeam('NYÍREGYHÁZA SPARTACUS FC',10).position.values[0]

8

Great

OK. we just have to do one thing. let's write a function to get the positions of all the turns

In [33]:
def teamPerRound(team, lastRound):
    
    pos = []
    
    for num in range(1, lastRound+1):
        posI = posTeam(team,num).position.values[0]
        pos.append(posI)
    
    
    return pd.DataFrame({'team':[f'{team}' for x in range(lastRound)], 'round':[x for x in range(1,lastRound+1)],'position':pos})

check

In [34]:
teamPerRound('NYÍREGYHÁZA SPARTACUS FC',17)

,team,round,position
0,NYÍREGYHÁZA SPARTACUS FC,1,16
1,NYÍREGYHÁZA SPARTACUS FC,2,19
2,NYÍREGYHÁZA SPARTACUS FC,3,19
3,NYÍREGYHÁZA SPARTACUS FC,4,17
4,NYÍREGYHÁZA SPARTACUS FC,5,17
5,NYÍREGYHÁZA SPARTACUS FC,6,16
6,NYÍREGYHÁZA SPARTACUS FC,7,16
7,NYÍREGYHÁZA SPARTACUS FC,8,15
8,NYÍREGYHÁZA SPARTACUS FC,9,12
9,NYÍREGYHÁZA SPARTACUS FC,10,8


done and great